# SPECFEM Users Workshop -- Day 3 (Oct. 7, 2022)

## Part 3: Seismic Imaging with SeisFlows + Pyatoa

In this notebook we will bring all of the previous material together run a full seismic inversion to update a starting model. 

-----------

### Relevant Information

>__NOTE:__ These instructions should be run from inside the Docker container, using Jupyter Lab. The Docker container should have the adjTomo toolkit installed (SeisFlows, Pyatoa, PySEP), as well as SPECFEM2D and SPECEFM3D compiled with MPI. 

**Relevant Links:** 
- Day 2 Slides !!! ADD THIS !!!
- [Today's Notebook](https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_2a_kernels.ipynb)
- Completed Notebook !!! ADD THIS !!!

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.


## 1) Background

!!! TO DO !!!
Potential topics:
- Nonlinear optimization algorithms
- Line searches
- Model perturbations

## 2) Setting Up 

As with Day 1, we will want to set up a clean working directory to run SPECFEM2D inside. This will help us preserve our cloned repository and reduce file clutter.

>__NOTE:__ We will be doing all our work in the directory /home/scoped/work_day_2. All the following cells assume that we are in this directory, so you must evaluate the '%cd' command to ensure that cells work as expected.

In [1]:
! mkdir /home/scoped/work/day_3
%cd /home/scoped/work/day_3

/home/scoped/work_day_2


In [3]:
! ls

bin  DATA  OUTPUT_FILES
